### Install, import and read files.

In [ ]:
%pip install pandas

In [148]:
import numpy as np
import pandas as pd 
# import s3fs

what_corpus = 'data_total'  # 'polnear', 'parc30'
what_type_files = 'test'

# filepath = 's3://sagemaker-studio-528576943967-ssf9zkrg3os/polnear-conll/prepared/'
filepath = '../' + what_corpus + '-conll/prepared/'

In [149]:
filename = what_corpus + '_preprocessed_' + what_type_files + '_noBIO.csv'
dataset = pd.read_csv(filepath + filename, sep=";")

In [150]:
# Loop over testfiles, loop over tokens, first look for CUE sequences and number them. Then find the closest SOURCE and CONTENTS sequences and give them the same number.
# If we loop over the file in dict format, we can change the dict easily during the loop.
# For finding SOURCE and CONTENT do an additional loop traveling the dict backwards and forwards at the same time with loop_index and -loop_index

# Additions
# Not just take the closest SOURCE and CONTENT, but also check if they are closer to another CUE.
# And at the end check and number the SOURCE and CONTENT labels without CUE.

In [151]:
class FilesGetter(object):
    
    def __init__(self, dataset):
        self.n_files = 1
        self.dataset = dataset
        self.empty = False
#         agg_func = lambda f: ' '.join(f["word"].values.tolist())
        agg_func = lambda f: [(w, t, f) for w, t, f in zip(f["word"].values.tolist(),
                                                           f["tag"].values.tolist(),
                                                           f["filename"].values.tolist())]
        self.grouped = self.dataset.groupby(["filename"]).apply(agg_func)
        self.files = [f for f in self.grouped]
    
    def get_next(self):
        try:
            f = self.grouped["Files: {}".format(self.n_tokens)]
            self.n_files += 1
            return f
        except:
            return None

In [152]:
files_getter = FilesGetter(dataset)

In [153]:
files = files_getter.files
print(type(files))

<class 'list'>


In [154]:
print(len(files))
# files[0:1]
# # [[('Washington', 'SOURCE'),
# #   ('Post', 'SOURCE'),
# #   ('back', 'CUE'),
# #   ('Hillary', 'CONTENT'),
# #   ('Clinton', 'CONTENT'),
# #   ('for', 'CONTENT'),
# #   ('president', 'CONTENT'),
# #   ('.', 'O'),
# #   ('Washington', 'O'),
# #   ('-lrb-', 'O'),
# #   ('AFP', 'O'),
# #   ('-rrb-', 'O'),
# #   ('--', 'O'),
# #   ('the', 'O'),
# #   ('Washington', 'O'),

184


In [155]:
# testlist = [1, 2, 3, 4, 5, 6]
# start_index = 2
# print(testlist[start_index])
# testlist.reverse()
# start_index = len(testlist) - start_index - 1
# print(start_index)
# print(testlist[start_index])

# for file_index, file in enumerate(file_temp[start_index:]):
#     print(file_index)

In [156]:
start_index_labels = 0  # = 2  # For BIO labels

def number_continuous_labels(file_temp, start_idx, label_nr, label_type, reverse=False):
    if reverse:
        file_temp.reverse()
        start_idx = (len(file_temp) - 1) - start_idx
        
    for file_index, file in enumerate(file_temp):
#         if file_index >= start_idx - 1:
        if file_index > start_idx:
#             print(file_index)
            label = file[1]
            if label == label_type:
                token = file[0]
                file_temp[file_index] = (token, label + '-' + str(label_nr))
            else:
                break

    # Reverse back once we're done
    if reverse:
        file_temp.reverse()
        
    return file_temp

# Find the CONTENT or SOURCE label for a CUE and add the number to the label
def number_label(token_idx, file_temp, label_nr, label_type):
    label_search_index = 1
    label_found_index = None
    
#     print(label_type)
#     print(file_temp)
    
    file_temp_len = len(file_temp)
    backwards_search_index = token_idx - label_search_index
    forwards_search_index = token_idx + label_search_index

#     print(file_temp_len)
#     print(backwards_search_index)
#     print(forwards_search_index)
#     print(len(file_temp[0:backwards_search_index]) > 0 or len(file_temp[forwards_search_index:file_temp_len]) > 0)

#     while len(file_temp[0:backwards_search_index]) >= 0 or len(file_temp[forwards_search_index:file_temp_len]) > 0:
    while backwards_search_index >= 0 or forwards_search_index < file_temp_len:
#         print(label_search_index)

        # Backwards loop
        if backwards_search_index >= 0:
            file_temp_item_backwards = file_temp[backwards_search_index]
#             print('backwards')
#             print(file_temp_item_backwards)
            label_backwards = file_temp_item_backwards[1]  # Label is 2nd in tuple
            if label_backwards[start_index_labels:] == label_type:
#                 print('label found')
                # Overwrite a possible content_found_index in the forward loop. 
                # We favor the backwards content_found_index over the forward loop.
                # If we find a content, break immediately from while loop
                label_found_index = backwards_search_index
                token_backwards  = file_temp_item_backwards[0]  # Token is 1st in tuple
                file_temp[label_found_index] = (token_backwards, label_backwards + '-' + str(label_nr))
#                 print(file_temp[label_found_index])
                file_temp = number_continuous_labels(file_temp, label_found_index, label_nr, label_type, reverse=True)
                break

        # Forwards loop
        if forwards_search_index < file_temp_len:
            file_temp_item_forwards = file_temp[forwards_search_index]
#             print('forwards')
#             print(file_temp_item_forwards)
            label_forwards = file_temp_item_forwards[1]  # Label is 2nd in tuple
            if label_forwards[start_index_labels:] == label_type: 
#                 print('label found')
                label_found_index = forwards_search_index
                token_forwards  = file_temp_item_forwards[0]  # Token is 1st in tuple
                file_temp[label_found_index] = (token_forwards, label_forwards + '-' + str(label_nr))
#                 print(file_temp[label_found_index])
                file_temp = number_continuous_labels(file_temp, label_found_index, label_nr, label_type)
                break

        label_search_index += 1 
        backwards_search_index = token_idx - label_search_index
        forwards_search_index = token_idx + label_search_index

        # Uncomment for debugging
#         if label_search_index == 1:  # One is first sentence with labels
#             break 

    return file_temp

In [159]:
files_labels_numbered = []
files_labels_max_label_nr = []
for file_index, file in enumerate(files):
#     file = file[0:30]  # Uncomment for debugging
    # Copy of the tokens in the file
    file_temp = file.copy()
#     file_temp_len = len(file_temp)
    label_nr = 1
    max_label_nr = 0
#     print('file')
#     print(file)
#     print(file_temp)
#     print(file_temp_len)
#     print(label_nr)
    
    # Loop over tokens to number labels
    cue_found_index = -1
    for token_idx, file_token in enumerate(file):
        label = file_token[1]  # Label is 2nd in tuple
        
#         print('token')
#         print(token_idx)
#         print(label)
        
        # If the label is CUE without a number, number the CUE first
        is_search_content_source = False
        if label[start_index_labels:] == 'CUE':
#             print('cue found')
#             print(cue_found_index)
#             print(token_idx > cue_found_index+1)

            # We are only handling continuous CUEs. If CUE is not continuous, then the label nr goes up 
            if token_idx > cue_found_index + 1:
                label_nr += 1
                is_search_content_source = True
                
            token = file_token[0]  # Token is 1st in tuple
            file_temp[token_idx] = (token, label + '-' + str(label_nr))
            cue_found_index = token_idx
#             print(file_temp[token_idx])
            
            if is_search_content_source:
#                 print('label_nr')
#                 print(label_nr)
                
#                 if label_nr == 3:
                # Now search backwards and forwards in file for CONTENT and SOURCE labels
                file_temp = number_label(token_idx, file_temp, label_nr, 'CONTENT')
                file_temp = number_label(token_idx, file_temp, label_nr, 'SOURCE')
                
            max_label_nr = label_nr
            
    # Append highest label nr found to list
    files_labels_numbered.append(file_temp)
    files_labels_max_label_nr.append(max_label_nr)
            
#     if file_index == 0:
#         break
                
print(len(files_labels_numbered))
# files_labels_numbered[1]

184


In [177]:
files_labels_numbered_padded = []
for file_idx, file in enumerate(files_labels_numbered):
    max_label_nr = files_labels_max_label_nr[file_idx]
    file_temp = file.copy()
    for token_idx, file_token in enumerate(file):
        token = file_token[0]
        label = file_token[1]
        if label != 'O':
            label_nr_str = label.replace('B','').replace('I','').replace('-','').replace('SOURCE','').replace('CONTENT','').replace('CUE','')
            # Try except
            if label_nr_str:
                label_nr = int(label_nr_str)
                lpad_string = "_ " * (max_label_nr - (max_label_nr - label_nr) - 1)
                rpad_string = " _" * (max_label_nr - label_nr)
                label = lpad_string + label + rpad_string
            else:
                # Replace quotation labels without a number by 'O'
                label = 'O'
        file_temp[token_idx] = (token, label)
    files_labels_numbered_padded.append(file_temp)
    
    if file_idx == 1:
        break
    
files_labels_numbered_padded[1]

[('CNN', '_ SOURCE-2 _ _ _ _ _ _ _ _ _ _ _ _ _ _'),
 ("'s", '_ SOURCE-2 _ _ _ _ _ _ _ _ _ _ _ _ _ _'),
 ('Jake', '_ SOURCE-2 _ _ _ _ _ _ _ _ _ _ _ _ _ _'),
 ('Tapper', '_ SOURCE-2 _ _ _ _ _ _ _ _ _ _ _ _ _ _'),
 ('on', '_ CONTENT-2 _ _ _ _ _ _ _ _ _ _ _ _ _ _'),
 ('Donna', '_ CONTENT-2 _ _ _ _ _ _ _ _ _ _ _ _ _ _'),
 ('Brazile', '_ CONTENT-2 _ _ _ _ _ _ _ _ _ _ _ _ _ _'),
 ('leak', '_ CONTENT-2 _ _ _ _ _ _ _ _ _ _ _ _ _ _'),
 ('to', '_ CONTENT-2 _ _ _ _ _ _ _ _ _ _ _ _ _ _'),
 ('Clinton', '_ CONTENT-2 _ _ _ _ _ _ _ _ _ _ _ _ _ _'),
 ('Campaign', '_ CONTENT-2 _ _ _ _ _ _ _ _ _ _ _ _ _ _'),
 (':', '_ CUE-2 _ _ _ _ _ _ _ _ _ _ _ _ _ _'),
 ('`', 'O'),
 ('unethical', 'O'),
 (',', 'O'),
 ("'", 'O'),
 ('`', 'O'),
 ('horrifying', 'O'),
 ('.', 'O'),
 ("'", 'O'),
 ('a', '_ _ SOURCE-3 _ _ _ _ _ _ _ _ _ _ _ _ _'),
 ('WikiLeaks', '_ _ SOURCE-3 _ _ _ _ _ _ _ _ _ _ _ _ _'),
 ('document', '_ _ SOURCE-3 _ _ _ _ _ _ _ _ _ _ _ _ _'),
 ('suggest', '_ _ CUE-3 _ _ _ _ _ _ _ _ _ _ _ _ _'),
 ('former', '_ _ C